In [6]:
from Tkinter import *
import tkFileDialog
from math import sin, cos, pi, fabs
from numpy import sign,arange
import sys
from PIL import Image, ImageTk
from boundaries import boundaries
from walls import walls
from Sphero import Sphero
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
        
# Addition of a Next button in the GUI window
def start():   # For the next button
    global pose, control, constraints, loop
    loop = not(loop)
    print "Lets start"
    while loop:
        pose = s.motion(pose,control,0.01)
        f=s.draw(pose)
        pose = s.check_collision(constraints, pose)
        print pose
        f=s.draw(pose[0])
#         robot_map(pose[0][0],pose[0][1],pose[1])
        root.update()
        pose = pose[0]

def stop():
    global loop
    loop = not(loop)
    print "Loop changed",loop
    
def boost():
    global control
    control[0] = control[0] + 50
    if control[0] <= 0:
        control[0] =0

def slow():
    global control
    control[0] = control[0] - 50
    if control[0] <= 0:
        control[0] =0
        
def robot_map(cx,cy,wall_type):
    if wall_type == 1:
        print "180 degree wall"
        ax1 = fig.add_axes([cx-r-ww, cy, ww, ww*3]) # GUI rectangle creation
        ax1.set_axis_off()
        ax2 = fig.add_axes([cx-r-ww, cy-ww*3, ww, ww*3])
        ax2.set_axis_off()
        cmap1 = mpl.cm.gray
        cmap2 = mpl.cm.Greys
        norm = mpl.colors.Normalize()
        cb1 = mpl.colorbar.ColorbarBase(ax1, cmap=cmap1,
                                       norm=norm,
                                       orientation='vertical')
        cb1 = mpl.colorbar.ColorbarBase(ax2, cmap=cmap2,
                                       norm=norm,
                                       orientation='vertical')
        plt.gcf().r.draw()
        
    elif wall_type == 2:
        print "270 degree wall"
        ax1 = fig.add_axes([cx, cy-r, ww*3, ww]) # GUI rectangle creation
        ax1.set_axis_off()
        ax2 = fig.add_axes([cx-ww*3, cy-r, ww*3, ww])
        ax2.set_axis_off()
        cmap1 = mpl.cm.gray
        cmap2 = mpl.cm.Greys
        norm = mpl.colors.Normalize()
        cb1 = mpl.colorbar.ColorbarBase(ax1, cmap=cmap1,
                                       norm=norm,
                                       orientation='horizontal')
        cb1 = mpl.colorbar.ColorbarBase(ax2, cmap=cmap2,
                                       norm=norm,
                                       orientation='horizontal')
        plt.gcf().r.draw()
        
    elif wall_type == 3:
        print "0 degree wall"
        ax1 = fig.add_axes([cx+r, cy, ww, ww*3]) # GUI rectangle creation
        ax1.set_axis_off()
        ax2 = fig.add_axes([cx+r, cy-ww*3, ww, ww*3])
        ax2.set_axis_off()
        cmap1 = mpl.cm.gray
        cmap2 = mpl.cm.Greys
        norm = mpl.colors.Normalize()
        cb1 = mpl.colorbar.ColorbarBase(ax1, cmap=cmap1,
                                       norm=norm,
                                       orientation='vertical')
        cb1 = mpl.colorbar.ColorbarBase(ax2, cmap=cmap2,
                                       norm=norm,
                                       orientation='vertical')
        plt.gcf().r.draw()
        
    elif wall_type == 4:
        print "90 degree wall"
        ax1 = fig.add_axes([cx, cy+r+ww, ww*3, ww]) # GUI rectangle creation
        ax1.set_axis_off()
        ax2 = fig.add_axes([cx-ww*3, cy+r+ww, ww*3, ww])
        ax2.set_axis_off()
        cmap1 = mpl.cm.gray
        cmap2 = mpl.cm.Greys
        norm = mpl.colors.Normalize()
        cb1 = mpl.colorbar.ColorbarBase(ax1, cmap=cmap1,
                                       norm=norm,
                                       orientation='horizontal')
        cb1 = mpl.colorbar.ColorbarBase(ax2, cmap=cmap2,
                                       norm=norm,
                                       orientation='horizontal')
        plt.gcf().r.draw()
        
    else:
        pass
        
if __name__ =="__main__":

    # Specifications of the world
    origin = (80,80)
    world_size = (498, 498)   # 750,750
    wall_width = 10
    wall_location =[[origin[0]+(world_size[0])/4,origin[1]+wall_width,origin[0]+(world_size[0])/4,origin[1]+wall_width+(world_size[1])*3/4],
                    [origin[0]+(world_size[0])/2,origin[1]+wall_width+(world_size[1])*1/4,origin[0]+(world_size[0])/2,origin[1]-wall_width+world_size[1]],
                    [origin[0]+(world_size[0])*3/4,origin[1]+wall_width,origin[0]+(world_size[0])*3/4,origin[1]+wall_width+(world_size[1])*3/4]] 
    
    #Initialization of boundary and walls
    sri = boundaries(world_size,wall_width) 
    mw = walls(wall_width,wall_location)
    
    # Intialization of GUI
    root = Tk()
    frame1 = Frame(root)
    frame1.pack()
    w = Canvas(frame1,width=700,height=700,bg="white")
    w.pack(side=LEFT)
    r = Canvas(frame1,width=700,height=700,bg="white")
    r.pack(side=RIGHT)
       
    # Creating world objects
    sri.draw(origin,w)
    mw.draw(wall_width,wall_location,w)
    parray = sri.boundary_grid(origin,w)
    warray = mw.wall_grid(wall_width,wall_location,w)
    
    # Creating Sphero robot
    s = Sphero(w)
    constraints = s.sphero_constraint(origin,wall_width,world_size,wall_location)
    
    pose = [276,225,3.25]
    control = [100,0]
    loop = 0
    frame2 = Frame(root)
    frame2.pack()
    start = Button(frame2,text="START", width=10,command=start)
    start.grid(row=0,column=0)
    stop = Button(frame2,text="STOP",width=10,command=stop)
    stop.grid(row=0,column=1)
    boost = Button(frame2,text="BOOST", width=10,command=boost)
    boost.grid(row=0,column=2)
    slow = Button(frame2,text="SLOW", width=10,command=slow)
    slow.grid(row=0,column=3)
    
    # Draw GUI
    root.mainloop()

Lets start
([275.00587032391945, 224.89180486546988, 3.25], 0)
([274.0117406478389, 224.78360973093976, 3.25], 0)
([273.01761097175836, 224.67541459640964, 3.25], 0)
([272.0234812956778, 224.56721946187952, 3.25], 0)
([271.02935161959726, 224.4590243273494, 3.25], 0)
([270.0352219435167, 224.35082919281928, 3.25], 0)
([269.04109226743617, 224.24263405828916, 3.25], 0)
([268.0469625913556, 224.13443892375903, 3.25], 0)
([267.0528329152751, 224.02624378922891, 3.25], 0)
([266.05870323919453, 223.9180486546988, 3.25], 0)
([265.064573563114, 223.80985352016867, 3.25], 0)
([264.07044388703343, 223.70165838563855, 3.25], 0)
([263.0763142109529, 223.59346325110843, 3.25], 0)
([262.08218453487234, 223.4852681165783, 3.25], 0)
([261.0880548587918, 223.3770729820482, 3.25], 0)
([260.09392518271125, 223.26887784751807, 3.25], 0)
([259.0997955066307, 223.16068271298795, 3.25], 0)
([258.10566583055015, 223.05248757845783, 3.25], 0)
([257.1115361544696, 222.9442924439277, 3.25], 0)
([256.11740647838

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Anaconda\lib\lib-tk\Tkinter.py", line 1486, in __call__
    return self.func(*args)
  File "<ipython-input-6-d9830db95cd5>", line 22, in start
    pose = s.check_collision(constraints, pose)
  File "Sphero.py", line 138, in check_collision
    if bingo > 0:
UnboundLocalError: local variable 'bingo' referenced before assignment


In [ ]:
import pyglet
music = pyglet.media.load("collision.mp3",streaming=False)
music.play()

In [ ]:
from Tkinter import Tk, Button
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2TkAgg

# plotting function: clear current, plot & redraw
def plot(x, y):
#     plt.clf()
    plt.plot(x,y)
    # just plt.draw() won't do it here, strangely
    plt.gcf().canvas.draw()

# just to see the plot change
plotShift = 0
def main():
    global plotShift

    x = np.arange(0.0,3.0,0.01)
    y = np.sin(2*np.pi*x + plotShift)
    plot(x, y)

    plotShift += 1

# GUI
root = Tk()

draw_button = Button(root, text="Plot!", command = main)
draw_button.grid(row=0, column=0)

# init figure
fig = plt.figure()

canvas = FigureCanvasTkAgg(fig, master=root)
toolbar = NavigationToolbar2TkAgg(canvas, root)
canvas.get_tk_widget().grid(row=0,column=1)
toolbar.grid(row=1,column=1)

root.mainloop()